In [3]:
%pip install networkx==3.0 scipy==1.10

  Using cached networkx-3.0-py3-none-any.whl.metadata (5.1 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 1.11.0
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11; 1.9.0 Requires-Python >=3.8,<3.12; 1.9.0rc1 Requires-Python >=3.8,<3.12; 1.9.0rc2 Requires-Python >=3.8,<3.12; 1.9.0rc3 Requires-Python >=3.8,<3.12; 1.9.1 Requires-Python >=3.8,<3.12
ERROR: Could not find a version that satisfies the requirement scipy==1.10 (from versions: 0.8.0, 0.9.0

In [4]:
import numpy as np
import pandas as pd 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import networkx as nx

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# --------------------------- Définition des Fonctions ---------------------------
def MRR_on_ranks(query_ranks):
    "Takes an array of ranks of the query set and returns the MRR of that query set."
    return np.mean(1 / np.array(query_ranks))

In [5]:
## Defining Figure Size:
sns.set(rc={"figure.figsize":(12, 6)})

#### Loading the Data

In [6]:
## Loading the Adjacency Matrix, node IDs, test dataset and Creating graph
M             = scipy.io.mmread('Data/tp-matrix.dgt').tocsr()
G             = nx.from_numpy_array(M, create_using=nx.DiGraph)
test_articles = pd.read_csv('Data/12-articles.csv')
node_ids_csv  = pd.read_csv('Data/tp-matrix-names.csv', sep="\s+")
node_ids_csv.rename(columns={"x": "id"}, inplace=True)

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\oussa\AppData\Local\Temp\ipykernel_12504\2424905474.py:5: SyntaxWarning: invalid escape sequence '\s'
  node_ids_csv  = pd.read_csv('Data/tp-matrix-names.csv', sep="\s+")


#### Showing the data

In [7]:
test_articles.head()

,id,titre,index
1,53e9aa4fb7602d97033bee00,Joint Energy Management and Resource Allocatio...,1594
2,53e9b098b7602d9703b03c4e,Qualitative organization of collections of sha...,11636
3,53e9b708b7602d97042a150f,Using the fuzzy multi-criteria decision making...,12215
4,53e9ba6ab7602d970468b7d5,Multi-Scale Adaptive Sampling with Mobile Agen...,12593
5,53e9ad6cb7602d9703755d9b,Randomized locality sensitive vocabularies for...,15089


In [8]:
node_ids_csv.head()

,id
1,53e99808b7602d970201b1fe
2,53e9b37bb7602d9703e5eb33
3,53e9b917b7602d9704500623
4,56d90f35dabfae2eee308269
5,53e9af87b7602d97039ce689


In [9]:
test_articles.iloc[0]

id                                53e9aa4fb7602d97033bee00
titre    Joint Energy Management and Resource Allocatio...
index                                                 1594
Name: 1, dtype: object

On observe un décalage entre les indices des sommets dans le CSV et dans le Graphe **G**

In [10]:
list(G.nodes())[0]

0

In [11]:
node_ids_csv.index[0]

1

In [12]:
## Getting the indices of test articles in the Graph
test_indices = node_ids_csv.index[node_ids_csv.id.isin(test_articles.id)] - 1 #The minus 1 is necessary because the first index is 1, in contrary to the first node that is 0
test_indices

Index([ 1593, 11635, 12214, 12592, 15088, 18538, 18644, 35303, 35352, 36564,
       50495, 50496],
      dtype='int64')

## Page Rank Global et personnalisé

In [13]:
# Computing the Global Pagerank via NetworkX
pr              = nx.pagerank(G, alpha=0.85)

#sorted_articles = sorted(pr.items(), key=lambda item: item[1], reverse=True)
sorted_articles = np.argsort(list(pr.values()))[::-1] # Sorting the articles by their pagerank

In [14]:
# Exemple qui montre qu'il faut rajouter +1
np.where(sorted_articles == 32)[0][0]+1

1

### Page rank global

In [15]:
# Calcul du MRR pour chacun des 12 articles. 
MRR_list = []
for i, test_index in enumerate(test_indices):
    reco_pertinente_liste = list(G.successors(test_index))
    rang_reco_pertinentes =np.array([np.where(sorted_articles == id)[0][0]+1 for id in reco_pertinente_liste])
    MRR = MRR_on_ranks(rang_reco_pertinentes)
    MRR_list.append(MRR)
    print("Le MRR de l'article `{0}` est de : {1:4f}".format(test_articles.iloc[i, 1], MRR))

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.002027
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.066947
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.002276
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.002594
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.000455
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.000145
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.002117
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.001798
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

### Page Rank Personnalisé 
Le lien à l'explication du Page Rank Thématique : [lien](https://www.youtube.com/watch?v=URaS1u-Murc&ab_channel=ArtificialIntelligence-AllinOne).

### Page Rank Personnalisé 

Assigner la valeur 2.0 à toutes les références adjacentes. Puis recaculer le MRR

In [16]:
# Calcul du MRR pour chacun des 12 articles.
MRR_list = []
for i, test_index in enumerate(test_indices):
    personalization = {node:2.0 for node in G.successors(test_index)}
    pr = nx.pagerank(G, alpha=0.85, personalization=personalization)
    sorted_articles = np.argsort(list(pr.values()))[::-1] # Sorting the articles by their pagerank
    reco_pertinente_liste = list(G.successors(test_index))
    rang_reco_pertinentes =np.array([np.where(sorted_articles == id)[0][0]+1 for id in reco_pertinente_liste])
    MRR = MRR_on_ranks(rang_reco_pertinentes)
    MRR_list.append(MRR)
    print("Le MRR de l'article `{0}` est de : {1:4f}".format(test_articles.iloc[i, 1], MRR))

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.232254
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.211296
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.211296
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.115960
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.244626
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.080108
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.111261
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.211296
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

#### Page Rank personnalisé avec validation croisée Leave one out : 

In [17]:
#### Page Rank personnalisé avec validation croisée Leave one out : 

# on creer une copie du graphe,

# on casse l'arrete entre le noeud et ses successeurs et on fait la validation croisee

MRR_list = []
for i, test_index in enumerate(test_indices):
    MRR_Copy_list = []
    for succ in G.successors(test_index):
        G_copy = G.copy()
        G_copy.remove_edge(test_index, succ)
        personalization = {node:2.0 for node in G_copy.successors(test_index)}
        pr = nx.pagerank(G_copy, personalization=personalization)
        sorted_articles = np.argsort(list(pr.values()))[::-1]
        reco_pertinente_liste = list(G_copy.successors(test_index))
        rang_reco_pertinentes =np.array([np.where(sorted_articles == id)[0][0]+1 for id in reco_pertinente_liste])
        MRR = MRR_on_ranks(rang_reco_pertinentes)
        MRR_Copy_list.append(MRR)
    MRR = np.mean(MRR_Copy_list)
    MRR_list.append(MRR)
    print("Le MRR de l'article `{0}` est de : {1:4f}".format(test_articles.iloc[i, 1], MRR))
print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.244626
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.221215
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.221215
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.118479
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.258601
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.081211
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.113556
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.221215
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

Obeservez-vous des différences entre le page rank global et personnalisé ?

## Calcul des moyennes de métriques et comparaison

Pour normaliser les mesures de telle sorte qu'un rappel de 100% des références donne le score maximal, en particulier dans le contexte où seuls certains articles ont un grand nombre de références, on doit adapter la procédure de calcul de la précision et du rappel pour tenir compte du nombre maximum de références pertinente par article. Si le nombre de références pertinentes est inférieur à k (le nombre de recommandations considérées pour la précision), le score doit être ajusté pour refléter ce maximum théorique.

In [18]:
normalized_precision_list_q2 = []
recall_list_q2 = []
k = 20

for i, test_index in enumerate(test_indices):
    personalization = {node:2.0 for node in G.successors(test_index)}
    pr = nx.pagerank(G, alpha=0.85, personalization=personalization)
    sorted_articles = np.argsort(list(pr.values()))[::-1] # Sorting the articles by their pagerank
    reco_pertinente_liste = list(G.successors(test_index))
    rang_reco_pertinentes =np.array([np.where(sorted_articles == id)[0][0]+1 for id in reco_pertinente_liste])
    # Calcul du Rappel
    total_relevant = len(reco_pertinente_liste) # Nombre total d'articles pertinents
    found_relevant = len(rang_reco_pertinentes[rang_reco_pertinentes <= k]) # Nombre d'articles pertinents retrouvés
    recall = found_relevant / total_relevant
    recall_list_q2.append(recall)
    
    # Normalisation de la précision
    normalized_precision = found_relevant / min(k, total_relevant)
    normalized_precision_list_q2.append(normalized_precision)

print(f"Précision normalisée moyenne pour les {k} premiers articles recommandés est de : {np.mean(normalized_precision_list_q2):4f}")
print(f"Rappel moyen pour les {k} premiers articles recommandés est de : {np.mean(recall_list_q2):4f}")


Précision normalisée moyenne pour les 20 premiers articles recommandés est de : 1.000000
Rappel moyen pour les 20 premiers articles recommandés est de : 0.868892


On observe que le page rank thématique donne de bien meilleurs résultats que le Page Rank Global !

## Aprroche Similarité Cosinus

In [19]:
cos_sim_test_adjmat    = cosine_similarity(M[test_indices], M)

In [20]:
## On remarque que la similarité cosinus entre deux vecteurs identiques est ~1
cos_sim_test_adjmat[:, test_indices]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [21]:
%%time
# Computing MRR for each article in test set:
print("\t\t\tRéstultats de l'approche Similarité Cosinus : \n")

MRR_list_q3 = []
for i, test_index in enumerate(test_indices):
    reco_pertinente_liste  = list(G.successors(test_index))
    cos_sim_test_adjmat[i, test_index] = 0 # On met à 0 les similarités entre le vecteur de test et lui même
    sorted_articles_cossim = np.argsort(cos_sim_test_adjmat[i])[::-1] # On trie les articles par similarité
    rang_reco_pertinentes  = np.array([np.where(sorted_articles_cossim == id)[0][0]+1 for id in reco_pertinente_liste])
    MRR = MRR_on_ranks(rang_reco_pertinentes)
    MRR_list_q3.append(MRR)
    print("Le MRR de l'article `{0}` est de : {1:4f}".format(test_articles.iloc[i, 1], MRR))

print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list_q3)))

			Réstultats de l'approche Similarité Cosinus : 

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.058316
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.001257
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.001830
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.010049
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.014697
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.019227
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.013581
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.008132
Le MRR de l'article `Dist

On remarque quand même une forte amélioration des résultats par rapport à la question 1. Mais que le pagerank thématique de la Q2 donne quand même de bien meilleurs résultats.

In [22]:
type(M)

scipy.sparse._csr.csr_matrix

#### Leave One Out

In [23]:
%%time
MRR_list_q3b = []
for i, test_index in enumerate(test_indices):
    rank_list = []
    reco_pertinente_liste = list(G.successors(test_index))
    for k in reco_pertinente_liste:
        M_copy = M.copy()
        # Leave one out
        cos_sim_test_adjmat = cosine_similarity(M_copy[test_index], M_copy)
        #On met à 0 les similarités entre le vecteur de test et lui même
        cos_sim_test_adjmat[:, test_index] = 0
        sorted_articles_cossim = np.argsort(cos_sim_test_adjmat[0])[::-1]
        rank = np.where(sorted_articles_cossim == k)[0][0]+1
        rank_list.append(rank)
    MRR = MRR_on_ranks(rank_list)
    MRR_list_q3b.append(MRR)
    print("Le MRR de l'article `{0}` est de : {1:4f}".format(test_articles.iloc[i, 1], MRR))
  
print("\n Le MRR moyen des 12 articles est de : {0:4f}".format(np.mean(MRR_list_q3)))

Le MRR de l'article `Joint Energy Management and Resource Allocation in Rechargeable Sensor Networks` est de : 0.058316
Le MRR de l'article `Qualitative organization of collections of shapes via quartet analysis` est de : 0.001257
Le MRR de l'article `Using the fuzzy multi-criteria decision making approach for measuring the possibility of successful knowledge management` est de : 0.001830
Le MRR de l'article `Multi-Scale Adaptive Sampling with Mobile Agents for Mapping of Forest Fires` est de : 0.010049
Le MRR de l'article `Randomized locality sensitive vocabularies for bag-of-features model` est de : 0.014697
Le MRR de l'article `Cache-Leakage Resilient OS Isolation in an Idealized Model of Virtualization` est de : 0.019227
Le MRR de l'article `Discrete tracking of parametrized curves` est de : 0.013581
Le MRR de l'article `Spatial template extraction for image retrieval by region matching.` est de : 0.008132
Le MRR de l'article `Distance sets for shape filters and shape recognition.`

## Approche LightGCN

In [24]:
%pip install -q torch==1.9.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.9.0 (from versions: 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==1.9.0


In [34]:
csv_file_path = "Data\\tp-matrix-names.csv"
output_path = "LightGCN-PyTorch\data\TP4\\User-list"

with open(csv_file_path, 'r') as csv_file:
    lines = csv_file.readlines()

with open(output_path, 'w') as txt_file:
    for line in lines[1:]:
        number = line.split('"')[1]
        txt_file.write(f"{number} {number}\n")

output_path


<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\oussa\AppData\Local\Temp\ipykernel_12504\3645441752.py:3: SyntaxWarning: invalid escape sequence '\d'
  output_path = "LightGCN-PyTorch\data\TP4\\User-list"


'LightGCN-PyTorch\\data\\TP4\\User-list'

In [45]:
import random

datasetNode = []
for node in G.nodes():
    datasetNode.append([node]+list(G.successors(node)))
    # ici je supprime les noeuds qui n'ont pas de successeurs
    if len(list(G.successors(node))) == 0:
        datasetNode.remove([node]+list(G.successors(node)))



random.shuffle(datasetNode)
train_test_split = 0.8
train_size = int(len(datasetNode) * train_test_split)
train_dataset = datasetNode[:train_size]
test_dataset = datasetNode[train_size:]

test_dataset.sort(key=lambda x: x[0])
train_dataset.sort(key=lambda x: x[0])

# On enregistre les train et les test datasets dans des fichiers textes
train_path = "LightGCN-PyTorch\data\TP4\\train.txt"
test_path = "LightGCN-PyTorch\data\TP4\\test.txt"

with open(train_path, 'w') as txt_file:
    for line in train_dataset:
        txt_file.write(" ".join([str(x) for x in line]) + "\n")

with open(test_path, 'w') as txt_file:
    for line in test_dataset:
        txt_file.write(" ".join([str(x) for x in line]) + "\n")

train_path, test_path

<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
C:\Users\oussa\AppData\Local\Temp\ipykernel_12504\4232129318.py:22: SyntaxWarning: invalid escape sequence '\d'
  train_path = "LightGCN-PyTorch\data\TP4\\train.txt"
C:\Users\oussa\AppData\Local\Temp\ipykernel_12504\4232129318.py:23: SyntaxWarning: invalid escape sequence '\d'
  test_path = "LightGCN-PyTorch\data\TP4\\test.txt"


('LightGCN-PyTorch\\data\\TP4\\train.txt',
 'LightGCN-PyTorch\\data\\TP4\\test.txt')

In [62]:
# On roule TensorBoard sur les derniers logs
%tensorboard --logdir="LightGCN-PyTorch\code\runs\04-08-14h40m06s--lgn" --host localhost --port 8088

UsageError: Line magic function `%tensorboard` not found.


![localImage](Screenshot%202024-04-08%20163148.png)